<a href="https://colab.research.google.com/github/dongukmoon/misc/blob/main/vllm_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# vLLM Tutorial: Quick Start on Google Colab
This guide walks you through setting up and running offline batched inference with vLLM on Google Colab.

## Running Offline Batched Inference on Colab

**Step 1: Install Required Packages**

Start by installing vLLM and other necessary packages using pip:

In [ ]:
!pip install vllm torch  triton

**Step 2: Set Up GPU Environment**

Google Colab offers free access to a Tesla T4 GPU (15GB RAM), which is sufficient for running vLLM. In this tutorial, we'll use the GPT-2 model and set the `gpu_memory_utilization` parameter to 0.5. This ensures that the key-value cache reserved for model inference does not exceed 50% of the total GPU memory. Without this setting, you might encounter out-of-memory errors due to the large key-value cache required for the maximum number of tokens supported by GPT-2.
With `gpu_memory_utilization`=0.5, the model weights will use about 0.24GB, and the key-value cache will require approximately 6.52GB of GPU memory.



**Step 3: Load the Model and Generate Outputs**

Below is the Python script to load the GPT-2 model, define prompts, and generate text outputs:

In [1]:
from vllm import LLM, SamplingParams

# Load a pre-trained Hugging Face model
model_name = "gpt2"  # Replace with your desired model

# Create an LLM.
#gpu_memory_utilization: The ratio (between 0 and 1) of GPU memory to
#reserve for the model weights, activations, and KV cache. Higher
#values will increase the KV cache size and thus improve the model's
#throughput. However, if the value is too high, it may cause out-of-
# memory (OOM) errors.
llm = LLM(model_name, gpu_memory_utilization = 0.5)

# Sample prompts.
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
# Create a sampling params object.
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

# Generate texts from the prompts. The output is a list of RequestOutput objects
# that contain the prompt, generated text, and other information.
outputs = llm.generate(prompts, sampling_params)
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

INFO 12-01 07:26:20 config.py:1861] Downcasting torch.float32 to torch.float16.
INFO 12-01 07:26:30 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='gpt2', speculative_config=None, tokenizer='gpt2', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=gpt2, num_scheduler_steps=1, chunked_prefill_enabled=False multi_step_stream_outputs=True, enable_prefix_caching=Fa

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

INFO 12-01 07:26:43 weight_utils.py:288] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 12-01 07:26:44 model_runner.py:1077] Loading model weights took 0.2378 GB
INFO 12-01 07:26:45 worker.py:232] Memory profiling results: total_gpu_memory=14.75GiB initial_memory_usage=0.37GiB peak_torch_memory=0.71GiB memory_usage_post_profile=0.40GiB non_torch_memory=0.15GiB kv_cache_size=6.52GiB gpu_memory_utilization=0.50
INFO 12-01 07:26:46 gpu_executor.py:113] # GPU blocks: 11861, # CPU blocks: 7281
INFO 12-01 07:26:46 gpu_executor.py:117] Maximum concurrency for 1024 tokens per request: 185.33x
INFO 12-01 07:26:51 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 12-01 07:26:51 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 

Processed prompts: 100%|██████████| 4/4 [00:00<00:00, 16.57it/s, est. speed input: 91.38 toks/s, output: 265.82 toks/s]

Prompt: 'Hello, my name is', Generated text: ' Scott. If you want to talk to me, you can. I am a'
Prompt: 'The president of the United States is', Generated text: ' a trained lawyer with experience in working with the vast majority of lawyers who are part'
Prompt: 'The capital of France is', Generated text: ' on the basis of the basis of the shared law. This is called the Common'
Prompt: 'The future of AI is', Generated text: ' becoming a bit more difficult. The big, big picture lies with AI. We'


**Step 4: Monitor GPU Memory Usage**

Use the nvidia-smi command to check GPU memory usage. As shown, approximately 7.5GB out of 15GB of GPU memory is used.

In [ ]:
!nvidia-smi

Sun Dec  1 04:51:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0              26W /  70W |   7525MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

**Sample Output**

The output of the script may look like this:

```
Prompt: 'Hello, my name is', Generated text: ' Scott. If you want to talk to me, you can. I am a'
Prompt: 'The president of the United States is', Generated text: ' a trained lawyer with experience in working with the vast majority of lawyers who are part'
Prompt: 'The capital of France is', Generated text: ' on the basis of the basis of the shared law. This is called the Common'
Prompt: 'The future of AI is', Generated text: ' becoming a bit more difficult. The big, big picture lies with AI. We'
```

**Step 5: Handle GPU Memory for Repeated Runs**

If you run the script again without restarting the Colab instance, you might encounter out-of-memory errors. To free up GPU memory, you can simply restart the Colab environment using the following command:

In [ ]:
import os
os._exit(00)

For more details, refer to the vLLM Documentation.